Datasets
- H&M
 - Goal: predict what article each customer will purchase for 7 days after training date ends.
 - Consists of articles.csv, customers.csv, transactions_train.csv
 - metrics: MAP@12 (Mean average percision at 12)

References:
- 


- Datasets
    - https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations
    - https://grouplens.org/datasets/movielens/

In [57]:
import numpy as np
import os
import sys
import pandas as pd
import tensorflow as tf

from tensorflow import keras

In [83]:
print(sys.version)
print(f"tensorflow v : {tf.__version__}")

3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
tensorflow v : 2.7.0


In [3]:
def convert_csv_to_parquet(folder_path, file):
    """
    Reads csv then convert to parquet file then remove original csv file
    
    parameters
    ---------
    folder_path : str, full path to folder that contain the file
    file : str, file_name with extension
    
    returns
    -------
    nothing
    
    """
    f_name = file.split(".")[0]
    file_path = os.path.join(folder_path, f_name)
    csv_file_path = file_path + ".csv"
    df = pd.read_csv(csv_file_path)
    df.to_parquet(file_path + ".parquet")
    os.remove(csv_file_path)

In [5]:
folder_path = '../datasets/h&m'
data_files = os.listdir(folder_path)
# for f in data_files:
#     convert_csv_to_parquet(folder_path, f)

In [6]:
article_df = pd.read_parquet(os.path.join(folder_path, "articles.parquet"))
customer_df = pd.read_parquet(os.path.join(folder_path, "customers.parquet"))
transaction_df = pd.read_csv(os.path.join(folder_path, "transactions_train.csv"))

In [37]:
print(transaction_df.shape)
transaction_df.dtypes

(31788324, 5)


t_dat                object
customer_id          object
article_id           object
price               float64
sales_channel_id      int64
dtype: object

In [41]:
article_df["article_id"] = article_df["article_id"].astype(str)
transaction_df["article_id"] = transaction_df["article_id"].astype(str)
transaction_df['t_dat'] = pd.to_datetime(transaction_df['t_dat'], format='%Y-%m-%d') 

In [ ]:
train_df = transaction_df

In [47]:
train_df = transaction_df.loc[(transaction_df['t_dat'] >= '2019-09-20') &
                              (transaction_df['t_dat'] < '2020-08-20')].copy()
test_df = transaction_df.loc[transaction_df['t_dat'] >= '2020-08-20'].copy()

print(train_df['t_dat'].min(), train_df['t_dat'].max())
print(test_df['t_dat'].min(), test_df['t_dat'].max())

In [56]:
train_df.article_id.unique()

array(['745014003', '812484002', '733098013', ..., '713824047',
       '933374001', '871558001'], dtype=object)

In [ ]:
class TwoTowerEmbeddingModel():
    """Model for learning query/item vector that will be mapped into
    same vector space, later for efficient vector computations.
    """
    def __init__(self):
        emb_dim = 32
        n_uniq_query = 1
        n_uniq_item = 2
        batch_size = 32
        
        self.query_model = tf.keras.layers.StringLookup(max_tokens=n_uniq_query)

In [142]:
uniq_query_ids = train_df.article_id.unique()
query_lookup = tf.keras.layers.StringLookup(vocabulary=uniq_query_ids)
query_vocab_size = len(uniq_query_ids) + 1

uniq_cust_ids = train_df.customer_id.unique()
customer_lookup = tf.keras.layers.StringLookup(vocabulary=uniq_cust_ids)
customer_vocab_size = len(uniq_cust_ids) + 1

In [ ]:
query_embeddings = tf.keras.layers.Embedding(query_vocab_size, emb_dim)
dense_1 = tf.keras.layers.Dense(256, activation='relu')(query_embeddings)
query_ output = tf.keras.layers.Dense(emb_dim, activation='relu')


item_embeddings = tf.keras.layers.Embedding(customer_vocab_size, emb_dim)
dense_1 = tf.keras.layers.Dense(256, activation='relu')(movie_embeddings)
item_output = tf.keras.layers.Dense(emb_dim, activation='relu')

- Stringlookup : for lookup operation, it is just like label encoder.
- n_uniq + 1 = vocab_size : to take into account out of vocab token. OOV token will have index = 0

In [108]:
uniq_article_ids = train_df.article_id.unique()
article_lookup = tf.keras.layers.StringLookup(vocabulary=uniq_article_ids)

# ??? why + 1?
article_vocab_size = len(uniq_article_ids) + 1

uniq_cust_ids = train_df.customer_id.unique()
customer_lookup = tf.keras.layers.StringLookup(vocabulary=uniq_cust_ids)
customer_vocab_size = len(uniq_cust_ids) + 1

In [106]:
train_df.head(2)

,t_dat,customer_id,article_id,price,sales_channel_id
16803901,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,745014003,0.020322,2
16803902,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,812484002,0.054220,2


In [105]:
dict(train_df.iloc[:3][['customer_id', 'article_id']])

{'customer_id': 16803901    0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...
 16803902    0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...
 16803903    0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...
 Name: customer_id, dtype: object,
 'article_id': 16803901    745014003
 16803902    812484002
 16803903    733098013
 Name: article_id, dtype: object}

- repeat()?
- brief review on tf.data.Dataset API

In [111]:
batch_size = 32

train_ds = (tf.data.Dataset
            .from_tensor_slices(dict(train_df[['customer_id', 'article_id']]))
            .shuffle(100_000)
            .batch(batch_size)
            .map(lambda x: {"article_id":article_lookup(x['article_id']), 
                           'customer_id':customer_lookup(x['customer_id'])})
            .repeat())

test_ds = (tf.data.Dataset
            .from_tensor_slices(dict(test_df[['customer_id', 'article_id']]))
            .batch(batch_size)
            .map(lambda x: {"article_id":article_lookup(x['article_id']), 
                           'customer_id':customer_lookup(x['customer_id'])})
            .repeat()) 

In [112]:
article_lookup.oov_token

'[UNK]'

In [124]:
all_article = article_lookup([article_lookup.oov_token] + list(article_lookup.input_vocabulary))

In [118]:
article_lookup.input_vocabulary

array(['745014003', '812484002', '733098013', ..., '713824047',
       '933374001', '871558001'], dtype=object)

In [115]:
article_vocab_size

66779

In [68]:
train_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
16803901,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,745014003,0.020322,2
16803902,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,812484002,0.054220,2
16803903,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,733098013,0.003373,2
16803904,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,802842001,0.028797,2
16803905,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,802842001,0.028797,2


In [66]:
train_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
16803901,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,745014003,0.020322,2
16803902,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,812484002,0.054220,2
16803903,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,733098013,0.003373,2
16803904,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,802842001,0.028797,2
16803905,2019-09-20,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,802842001,0.028797,2


In [63]:
print(article_vocab_size)

66779



- recall@k : n_hit/len(targets), n_hit = set(pred) & set(targets)  
- MAP@k : [0,1,1] -> prec@k = [0, 1/2, 2/3] -> AP@k = (1/k)[1/2+2/3] -> MAP@K = mean of APs(multiple lists).

In [125]:
transaction_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
